In [1]:
import csv
import os
from functools import partial
import json

import geopandas as gpd
import matplotlib.pyplot as plt

from gerrychain import (
    Election,
    Graph,
    MarkovChain,
    Partition,
    accept,
    constraints,
    updaters,
)
from gerrychain.metrics import efficiency_gap, mean_median
from gerrychain.proposals import recom
from gerrychain.updaters import cut_edges
from gerrychain.tree import recursive_tree_part

In [2]:
path = "./CompleteCOwithDemographics.shp"

In [3]:
graph = Graph.from_file(path)

In [4]:
df = gpd.read_file(path)

In [5]:
newdir = "./TryingtoShowPictures/"
os.makedirs(os.path.dirname(newdir + "init.txt"), exist_ok=True)
with open(newdir + "init.txt", "w") as f:
    f.write("Created Folder")

In [6]:
unique_label = "VTDSD"
census_pop = "Sum_POP2_1"
pop_est = "Sum_block_"
congress = "CD116FP"
senate = "SLDUST"
house = "SLDLST"
county_col = "COUNTYFP"

num_elections = 5
election_names = [
    "AG18",
    "SEC18",
    "TRE18",
    "GOV18",
    "REG18",
]
election_columns = [
    ["AG_DEM18", "AG_REP18"],
    ["SEC_DEM18", "SEC_REP18"],
    ["TRE_DEM18", "TRE_REP18"],
    ["GOV_DEM18", "GOV_REP18"],
    ["REG_DEM18", "REG_REP18"],
]

num_districts = 7

In [ ]:
def num_splits(partition, level = county_col):
    df["current"] = df[unique_label].map(dict(partition.assignment))
    splits = sum(df.groupby(level)["current"].nunique() > 1)
    return splits

In [7]:
updaters = {
    "population": updaters.Tally(census_pop, alias="population"),
    "cut_edges": cut_edges,
}

elections = [
    Election(
        election_names[i],
        {"Democratic": election_columns[i][0], "Republican": election_columns[i][1]},
    )
    for i in range(num_elections)
]

election_updaters = {election.name: election for election in elections}

updaters.update(election_updaters)

In [8]:
enacted_partition = Partition(graph, senate, updaters)
ideal_population = sum(enacted_partition['population'].values()) / len(enacted_partition)

In [12]:
cddict = recursive_tree_part(graph, range(num_districts), ideal_population, census_pop, .001, 1)

df['random'] = df.index.map(cddict)

df.plot(column = 'random', cmap = 'jet')
plt.savefig(newdir + 'random.png')
plt.close()

with open(newdir + 'random.json', 'w') as jf1:
    json.dump(cddict, jf1)

In [15]:
df.plot(column = senate, cmap = 'jet')
plt.savefig(newdir + 'senate.png')
plt.close()

df.plot(column = house, cmap = 'jet')
plt.savefig(newdir + 'house.png')
plt.close()

df.plot(column = congress, cmap = 'jet')
plt.savefig(newdir + 'congress.png')
plt.close()